<a href="https://colab.research.google.com/github/jgmsgabriel/formacao_LLM/blob/main/Joao_Curso_LLMs_Agentes_e_Tools_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers einops accelerate bitsandbytes
!pip install -q langchain langchain_community langchain-huggingface langchainhub langchain_chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 95.2 MB/s eta 0:00:

In [2]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=62d33170704b47d4c12657cf77995dd8ca7284068f64ebe1f770f384cea0f9cc
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [3]:
from langchain_core.tools import Tool
from langchain.tools import WikipediaQueryRun, BaseTool
from langchain_community.utilities import WikipediaAPIWrapper

In [6]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=3000))

In [7]:
print(wikipedia.run("Deep Learning"))

Page: Deep learning
Summary: Deep learning is a subset of machine learning that focuses on utilizing neural networks to perform tasks such as classification, regression, and representation learning. The field takes inspiration from biological neuroscience and is centered around stacking artificial neurons into layers and "training" them to process data. The adjective "deep" refers to the use of multiple layers (ranging from three to several hundred or thousands) in the network. Methods used can be either supervised, semi-supervised or unsupervised.
Some common deep learning network architectures include fully connected networks, deep belief networks, recurrent neural networks, convolutional neural networks, generative adversarial networks, transformers, and neural radiance fields. These architectures have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate sc

In [8]:
print(wikipedia.run("Oscar Niemeyer"))

Page: Oscar Niemeyer
Summary: Oscar Ribeiro de Almeida Niemeyer Soares Filho (15 December 1907 – 5 December 2012), known as Oscar Niemeyer (Brazilian Portuguese: [ˈoskaʁ ni.eˈmajeʁ]), was a Brazilian architect considered to be one of the key figures in the development of modern architecture. Niemeyer was best known for his design of civic buildings for Brasília, a planned city that became Brazil's capital in 1960, as well as his collaboration with other architects on the headquarters of the United Nations in New York. His exploration of the aesthetic possibilities of reinforced concrete was highly influential in the late 20th and early 21st centuries.
Both lauded and criticized for being a "sculptor of monuments", Niemeyer was hailed as a great artist and one of the greatest architects of his generation by his supporters. He said his architecture was strongly influenced by Le Corbusier, but in an interview, assured that this "didn't prevent [his] architecture from going in a different 

In [9]:
print(wikipedia.run("Oscars 2024"))

Page: 96th Academy Awards
Summary: The 96th Academy Awards ceremony, presented by the Academy of Motion Picture Arts and Sciences (AMPAS), took place on March 10, 2024, at the Dolby Theatre in Hollywood, Los Angeles. During the gala, the AMPAS presented Academy Awards (commonly referred to as Oscars) in 23 categories honoring films released in 2023. The ceremony, televised in the United States by ABC, was produced by Raj Kapoor and Katy Mullan, and was directed by Hamish Hamilton. Comedian Jimmy Kimmel hosted the show for the fourth time. He first presided over the 89th ceremony held in 2017, and had most recently hosted the previous year's ceremony.
In related events, the Academy held its 14th Governors Awards ceremony at the Ray Dolby Ballroom of the Ovation Hollywood complex in Hollywood, on January 9, 2024. The Academy Scientific and Technical Awards were presented by host Natasha Lyonne on February 23, 2024, in a ceremony at the Academy Museum of Motion Pictures in Los Angeles.
Op

## Definição da Tool

In [10]:
wikipedia_tool = Tool(name = "wikipedia",
                      description="You must never search for multiple concepts at a single step, you need to search one at a time. When asked to compare two concepts for example, search for each one individually.",
                      func=wikipedia.run)

## Criando Tool Customizada

In [12]:
def current_day(*args, **kwargs):
    from datetime import date

    dia = date.today()
    dia = dia.strftime("%d/%m/%Y")
    return dia

In [13]:
date_tool = Tool(name="Day", func = current_day,
                 description = "Use when you need to know the current date")

In [14]:
tools = [wikipedia_tool, date_tool]

## ReAct

In [15]:
import getpass
import os
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [16]:
from langchain import hub
prompt = hub.pull("hwchase17/react")

In [17]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

## Carregando a LLM

In [18]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace

In [19]:
llm = HuggingFaceEndpoint(repo_id="meta-llama/Meta-Llama-3-8B-Instruct", temperature=0.1)

<ipython-input-19-2a9605fd5d2c>:1: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceEndpoint(repo_id="meta-llama/Meta-Llama-3-8B-Instruct", temperature=0.1)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

In [20]:
from langchain.agents import AgentExecutor, create_react_agent

agent = create_react_agent(llm = llm, tools = tools, prompt = prompt)

In [21]:
agent_executor = AgentExecutor.from_agent_and_tools(agent = agent, tools = tools,
                                                    verbose = True, handling_parsing_errors = True)

In [22]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [28]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


In [29]:
resp = agent_executor.invoke({"input": "Em que dia estamos?"})



> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/inference/_client.py:2245: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-3-8B-Instruct (Request ID: zy6MG5)

TooManyRequests: Please log in or use a HF access token

In [ ]:
resp = agent_executor.invoke({"input": "Qual a população de Paris?"})